In [ ]:
import pandas as pd
import numpy as np
import matplotlib
# matplotlib.use('Agg')
%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt
import math

from sklearn.linear_model import LassoCV
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import statsmodels.api as sm
import copy
import os
import shutil

%matplotlib inline
matplotlib.use('module://ipykernel.pylab.backend_inline')
import matplotlib.pyplot as plt


In [ ]:
def add_time_difference(df_source, i):
    assignment = {}
    assignment[f"time_difference.0.{i}"] = (pd.to_datetime(df_source[f"date_visit.{i}"]) - pd.to_datetime(df_source["date_visit.0"])).dt.total_seconds() / (24 * 3600)
    df_source = df_source.assign(**assignment)
    return df_source

In [ ]:
n_samples =None
memory_experiment = "pm"
compute_simple_models = False


binary_smoke = False

second_visit = 2
lasso_epsilon = 0.01
p_val = 0.05
age_limit = 0
gender = None
iqr_coefficient = None # if None then no standrat removal of outliers

keep_ldl_hdl = True # if lasso selects only one of them keep the second anyway; they are correlated but have different influence
quadr_on_log_if_skewed = True

HSCD = 'HC1' # if none than the model builder will be heteroscedascisity unaware

input_dir = "/projects/prime/ukbb/preprocessed_data_2024"
imputed_dir = f"/projects/prime/ukbb/preprocessed_data_2024/imputations/ukbb_imputed_{memory_experiment}_sg_0_{second_visit}_rf/ukbb_imputed_{memory_experiment}_sg_0_{second_visit}_rf_instance_1"
path_imputed =  f"{imputed_dir}/ukbb_imputed_{memory_experiment}_sg_0_{second_visit}_rf_instance_1.csv"
input_filenames = {
    "diet": "diet_2024_bradbury_march.csv",
    "alcohol": "alcohol_2024_maart.csv",
    "diagnoses": "diagnoses_2024.csv",
    "lifestyle": "participants_age_date_smoke_MET_2024_jan.csv",
    "medications": "participants_medications_2024_jan.csv",
    "risk_factors": "blood_count_biochemistry_pressure_maart_2024.csv",
    "sociodemo": "edu_job_marriage_2024.csv",
    "cognition": "cognitive_2024_maart.csv",
    "metrics": "body_size_measures_gender_dates_2024_jan.csv"
  }

output_dir =f"/projects/prime/ukbb/results_2024/{memory_experiment}_sg_0_{second_visit}_no_interactions_quadr_hdl_ldl_remove_skewed_originals_27_05_nutr_only"
if HSCD is not None:
  output_dir = f"{output_dir}_{HSCD}"

if gender is None:
  output_dir = f"{output_dir}/"
else:
  output_dir = f"{output_dir}_gender_{gender}/"
if os.path.exists(output_dir):
   shutil.rmtree(output_dir)
    # The directory does not exist, create it
os.makedirs(output_dir)

In [ ]:
if memory_experiment == "nm":
    memory_experiment_full_name = "num_memory_max_digits_remembered_correctly"
else:
    memory_experiment_full_name = "pairs_matching_sum_incorrect"

In [ ]:
'''
predictors = ["gender.0", f"time_difference.0.{second_visit}", "Diabetes_2.0",  "Dyslipidemia.0", "Depression.0", "Hypertension.0", 
  "age_years.0", "education_level.0", "marital_status.0", "pp_smoke_catgeory.0", "MET_score.0", "waist_cm.0",  "bmi_kg_m2.0", 
  "Syst_bp.0", "Diast_bp.0", "ldl_conv.0", "hdl_conv.0", "Triglycerides_conv.0", "HbA1c_conv.0",  "Pl_glucose_conv.0",   "Albumin.0", 
  "Leukocyte_count.0", "C_reactive_protein.0",
   "oily_fish_gpday_bradbury.0", "white_fish_gpday_bradbury.0", 
   "red_meat_bradbury_gpd.0", "poultry_gpday_bradbury.0",
   "processed_meat_gpday_bradbury.0", "veg_gpday_bradbury.0",
   "fruit_gpday_bradbury.0",
    "cereals_gpday_bradbury.0", "bread_gpday_bradbury.0", "cheese_gpday_bradbury.0",
    "milk_gpday_bradbury.0", "tea_gpday_bradbury.0",
    "red_wine_gpd.0", "white_wine_gpd.0", "fortified_gpd.0", "beer_cider_gpd.0", "spirits_gpd.0",
   "aspirin.0",  "anxiety_tr.0", "pain_tr.0", "TAZD_Thiazide.0",  
   "loop_diuretics.0",  "potassium_diuretics.0", "beta_blockers.0", "calcium_antagonists.0", 
   "ARA_II_Antagonists_of_angiotensin_II_receptors.0", "IECA_Angiotensin_converting_enzyme_inhibitors.0",
   "Other_Hypotensive.0","hypochol_statins.0", "hypochol_others.0", "insulin.0", "sulfonylurea.0", "thiazolidinediones.0", "non_sulfonylurea_insulin_secretagogues.0", "metformin_category.0", "vitamins_minerals.0"]
   '''

In [ ]:
predictors = ["gender.0", "age_years.0", "education_level.0", f"time_difference.0.{second_visit}",
   "oily_fish_gpday_bradbury.0", "white_fish_gpday_bradbury.0", 
   "red_meat_bradbury_gpd.0", "poultry_gpday_bradbury.0",
   "processed_meat_gpday_bradbury.0", "veg_gpday_bradbury.0",
   "fruit_gpday_bradbury.0",
    "cereals_gpday_bradbury.0", "bread_gpday_bradbury.0", "cheese_gpday_bradbury.0",
    "milk_gpday_bradbury.0", "tea_gpday_bradbury.0",
    "red_wine_gpd.0", "white_wine_gpd.0", "fortified_gpd.0", "beer_cider_gpd.0", "spirits_gpd.0"]

In [ ]:
predictors_to_plot = [f"time_difference.0.{second_visit}", 
  "age_years.0", "MET_score.0", "waist_cm.0",  "bmi_kg_m2.0", 
  "Syst_bp.0", "Diast_bp.0", "ldl_conv.0", "hdl_conv.0", "Triglycerides_conv.0", "HbA1c_conv.0",  "Pl_glucose_conv.0",   "Albumin.0", 
  "Leukocyte_count.0", "C_reactive_protein.0",
   "oily_fish_gpday_bradbury.0", "white_fish_gpday_bradbury.0", 
   "red_meat_bradbury_gpd.0", "poultry_gpday_bradbury.0",
   "processed_meat_gpday_bradbury.0", "veg_gpday_bradbury.0",
   "fruit_gpday_bradbury.0",
    "cereals_gpday_bradbury.0", "bread_gpday_bradbury.0", "cheese_gpday_bradbury.0",
    "milk_gpday_bradbury.0", "tea_gpday_bradbury.0",
    "red_wine_gpd.0", "white_wine_gpd.0", "fortified_gpd.0", "beer_cider_gpd.0", "spirits_gpd.0"
    ]

In [ ]:
outliers_standart = ["Syst_bp.0", "Diast_bp.0"]

In [ ]:
simple_predictors =["age_years.0", "gender.0", "education_level.0", f"log_time_difference.0.{second_visit}"]
if (gender is not None) and ("gender.0") in simple_predictors:
    simple_predictors.remove("gender.0") 

In [ ]:
outcome_components = [f"{memory_experiment_full_name}.0", f"{memory_experiment_full_name}.{second_visit}", 
            "snap_game_true_pos_rt_avrg.0", f"snap_game_true_pos_rt_avrg.{second_visit}"]

In [ ]:
log_features = [f"time_difference.0.{second_visit}", "age_years.0", "MET_score.0", 
  "Triglycerides_conv.0", "HbA1c_conv.0",  "Pl_glucose_conv.0",   
  "Leukocyte_count.0", "C_reactive_protein.0",
  "red_wine_gpd.0", "white_wine_gpd.0", "fortified_gpd.0", "beer_cider_gpd.0", "spirits_gpd.0"
    ]

In [ ]:
quadr_features = ["MET_score.0", "waist_cm.0",  "bmi_kg_m2.0", 
  "Syst_bp.0", "Diast_bp.0", "ldl_conv.0", "hdl_conv.0", "Triglycerides_conv.0", "HbA1c_conv.0",  "Pl_glucose_conv.0",   "Albumin.0", 
  "Leukocyte_count.0", "C_reactive_protein.0",
   "oily_fish_gpday_bradbury.0", "white_fish_gpday_bradbury.0", 
   "red_meat_bradbury_gpd.0", "poultry_gpday_bradbury.0",
   "processed_meat_gpday_bradbury.0", "veg_gpday_bradbury.0",
   "fruit_gpday_bradbury.0",
    "cereals_gpday_bradbury.0", "bread_gpday_bradbury.0", "cheese_gpday_bradbury.0",
    "milk_gpday_bradbury.0", "tea_gpday_bradbury.0"
   #  "red_wine_gpd.0", "white_wine_gpd.0", "fortified_gpd.0", "beer_cider_gpd.0", "spirits_gpd.0"
    ]

In [ ]:
'''outliers = {
    "ldl_conv.0": [0, 290],
    "Syst_bp.0": [0, 220],
}'''
outliers = {}

In [ ]:

interaction_lists = []
interaction_pairs = []

# very skewed:
remove_original_features = ["MET_score.0", "Triglycerides_conv.0", "Leukocyte_count.0", "C_reactive_protein.0",
                            "red_wine_gpd.0", "white_wine_gpd.0", "fortified_gpd.0", "beer_cider_gpd.0", "spirits_gpd.0"
                            ]

In [ ]:
all_needed = ["f.eid"] + predictors + outcome_components
for i in range(1,4):
    if f"time_difference.0.{i}" in predictors: # will be calculate from dates
        all_needed.remove(f"time_difference.0.{i}")
        all_needed = all_needed + [f"date_visit.{i}"] 
        if "date_visit.0" not in all_needed:
             all_needed = all_needed + ["date_visit.0"] 

In [ ]:
# load raw data set and select all not NA
df_index = {}
first = True
for label, filename in input_filenames.items():
    print(label)
    full_path = f"{input_dir}/{filename}"
    
    data_columns = pd.read_table(full_path, nrows=1, sep=',').columns
    
    needed_fields = data_columns[data_columns.isin(all_needed)]
    
    if label == "medications":
        needed_fields = needed_fields.tolist() + ["alpha_glucosidase_inhibitors.0"] 
        
    df_index[label] = pd.read_table(full_path, sep=',', usecols=needed_fields, dtype=str, nrows=n_samples)
    
    
    print(len(df_index[label]))
    
    for clmn in needed_fields:
        if clmn == "f.eid" or clmn.startswith("date_visit"):
            continue
        df_index[label][clmn] = df_index[label][clmn].astype(float)
        
    
    if first:
        df = df_index[label]
        first = False
    else:
        df = pd.merge(df,df_index[label])
        
    print(len(df))



# remove NA from the check up data set 
df = df.dropna(axis="rows")
print(f"cleaned check up db has {len(df)} rows")

df= df.loc[df["alpha_glucosidase_inhibitors.0"] != 1]
print("removed all participants with alpha_glucosidase_inhibitors.0 == 1:")  
df = df.drop("alpha_glucosidase_inhibitors.0", axis = 1)
print(len(df))



In [ ]:
# load imputed data set
df_imputed = pd.read_table(path_imputed, sep=',', usecols=all_needed, dtype=str, nrows=n_samples)

In [ ]:
# make all columns if the imputed except dates, float
for clmn in df_imputed.columns:
    if clmn == "f.eid" or clmn.startswith("date_visit"):
        continue
    df_imputed[clmn] = df_imputed[clmn].astype(float)

In [ ]:
# filter both data set by age limits (which can be zero)
if "age_years.0" in df.columns:
    df= df[df["age_years.0"].ge(age_limit)]
    print("removed all participants younger age_limits no NA:") 
    print(len(df))

if "age_years.0" in df_imputed.columns:
    df_imputed= df_imputed[df_imputed["age_years.0"].ge(age_limit)]
    print("removed all participants younger age_limits imputed:") 
    print(len(df_imputed))

In [ ]:
# filter both data sets by gender if required
if gender is not None and "gender.0" in df.columns and "gender.0" in df_imputed.columns:
    df= df.loc[df["gender.0"].eq(gender)]
    df.drop(columns=['gender.0'])
    print(f"only participants with gender {gender} are left in the not na data set: ") 
    
    print(len(df)) 
    
    df_imputed= df_imputed.loc[df_imputed["gender.0"].eq(gender)]
    df_imputed.drop(columns=['gender.0'])
    print(f"only participants with gender {gender} are left in the omputed data set: ") 
    print(len(df_imputed))

In [ ]:
# add to both data data sets time difference column
for i in range(1,4):
    if f"time_difference.0.{i}" in predictors:
        df = add_time_difference(df, i)   
        df_imputed = add_time_difference(df_imputed, i)

In [ ]:
# add to both data sets binarised smoke score 
if binary_smoke:
    
    ind_replace = predictors.index("smoking_score.0")
    predictors[ind_replace] = "binary_smoking_score.0"
    
    assignment = {}
    assignment["binary_smoking_score.0"] =np.where(df["smoking_score.0"] == 2,1, 0) 
    df = df.assign(**assignment)

    assignment = {}
    assignment["binary_smoking_score.0"] =np.where(df_imputed["smoking_score.0"] == 2,1, 0) 
    df_imputed = df_imputed.assign(**assignment)



In [ ]:
# remove from both data sets customized outliers
for feat, out_ in outliers.items():
   df_imputed= df_imputed[(df_imputed[feat] >= out_[0]) & (df_imputed[feat] <= out_[1])]
   print(f"data set imputed after removing customized outliers: {len(df_imputed)}")
   df= df[df[feat] >= out_[0] & df[feat] <= out_[1]]
   print(f"data set no na after removing customized outliers: {len(df)}")

In [ ]:
# function to add log features 
def add_logs(df_, log_features, outcome_components, descr):
    print(f"adding logarithmic features to {descr}")
    assignment = {}
    log_cols = []

    for log_feature in log_features:
        if log_feature in df_.columns:
            new_fld = f"log_{log_feature}"
            assignment[new_fld] = np.log1p(df_[log_feature])
            log_cols = log_cols + [new_fld]
            print(new_fld)
    
    for i in range(0,4):
        
        if f"pairs_matching_sum_incorrect.{i}" in outcome_components:
            new_fld = f"log_pairs_matching_sum_incorrect.{i}"
            if new_fld not in assignment:
                assignment[new_fld] = np.log1p(df_[f"pairs_matching_sum_incorrect.{i}"])
                print(new_fld)
                
        
        if f"pairs_matching_incorrect_in_round_1.{i}" in outcome_components:
            new_fld_round = f"log_pairs_matching_incorrect_in_round_1.{i}"
            if new_fld_round not in assignment:
                assignment[new_fld_round] = np.log1p(df_[f"pairs_matching_incorrect_in_round_1.{i}"])  
                print(new_fld) 

    df_ = df_.assign(**assignment)
    return df_, log_cols

In [ ]:
df, log_cols =  add_logs(df, log_features, outcome_components, "check up db with no NA")
df_imputed, _  =  add_logs(df_imputed, log_features, outcome_components,"df with imputed")

In [ ]:
# function to quadr features 
def add_squares(df_, q_features, descr, skewed_only_on_log, skewed_features):
    print(f"adding quadratic features to {descr}")
    assignment = {}
    quadr_cols = []
    
    for feat in df_.columns:
        for q_feat in q_features:
            if q_feat in feat:
                if not skewed_only_on_log or ("log_" in feat) or not (feat in skewed_features):
                    new_fld = f"quadr_{feat}"
                    assignment[new_fld] = np.square(df_[feat])
                    quadr_cols = quadr_cols + [new_fld]
                    print(new_fld)
    
    

    df_ = df_.assign(**assignment)
    return df_, quadr_cols

In [ ]:
df, quad_cols =  add_squares(df, quadr_features, "check up db with no NA", quadr_on_log_if_skewed, remove_original_features)
df_imputed, _  =  add_squares(df_imputed, quadr_features, "df with imputed", quadr_on_log_if_skewed, remove_original_features)

In [ ]:
# function for removing outliers via iqr approach
def remove_outliers_iqr(df_, iqr_cf, df_name):
    if iqr_cf is not None:
        
        lower_bound = {}
        upper_bound = {}
        
        for feat  in outliers_standart:
            print(f"{feat}")
            
            Q1 = df_[feat].quantile(0.25)
            Q3 = df_[feat].quantile(0.75)
            
            IQR = Q3-Q1
            lower_bound[feat] = Q1 - iqr_cf * IQR
            upper_bound[feat] = Q3 + iqr_cf * IQR
            
            if feat in log_features:
                feat=f"log_{feat}"
                print(f"logarithimic outlier {feat}")
            
                Q1 = df_[feat].quantile(0.25)
                Q3 = df_[feat].quantile(0.75)
                
                IQR = Q3-Q1
                lower_bound[feat] = Q1 - iqr_cf * IQR
                upper_bound[feat] = Q3 + iqr_cf * IQR
            
            
                
         
            
        for feat in lower_bound:
            df_= df_[(df_[feat] >= lower_bound[feat]) & (df_[feat] <= upper_bound[feat])]
            
        print(f"data set {df_name} after removing outliers: {len(df_)}")
    return(df_)

In [ ]:
# removing outliers via iqr approach
df = remove_outliers_iqr(df, iqr_coefficient, "no na")
df_imputed = remove_outliers_iqr(df_imputed, iqr_coefficient, "imputed")
        

In [ ]:
# function which adds the column with the RV scores to the data set
def adding_rv_scores(df_source, mem_experiment_full, repeat):
    assignment = {}
    weight_mem = 1
    if "pairs_matching" in mem_experiment_full:
        weight_mem = -1
        
    sum_mem_0 = df_source[f"{mem_experiment_full}.0"].sum()
    mem_mean_0 = df_source[f"{mem_experiment_full}.0"].mean()
    mem_std_0 = df_source[f"{mem_experiment_full}.0"].std()
    assignment[f'z_{mem_experiment_full}.0'] = (df_source[f"{mem_experiment_full}.0"] - mem_mean_0)/mem_std_0
    
    sum_mem_repeat = df_source[f"{mem_experiment_full}.{repeat}"].sum()
    mem_mean_repeat = df_source[f"{mem_experiment_full}.{repeat}"].mean()
    mem_std_repeat = df_source[f"{mem_experiment_full}.{repeat}"].std()
    assignment[f'z_{mem_experiment_full}.{repeat}'] = (df_source[f"{mem_experiment_full}.{repeat}"] - mem_mean_0)/mem_std_0 
    
    sum_sg_rt_mean_0 = df_source["snap_game_true_pos_rt_avrg.0"].sum()
    sg_rt_mean_0 = df_source["snap_game_true_pos_rt_avrg.0"].mean()
    sg_rt_std_0 = df_source["snap_game_true_pos_rt_avrg.0"].std()
    assignment['z_snap_game_true_pos_rt_avrg.0'] = (df_source["snap_game_true_pos_rt_avrg.0"] - sg_rt_mean_0)/sg_rt_std_0

    sum_sg_rt_mean_repeat = df_source[f"snap_game_true_pos_rt_avrg.{repeat}"].sum()
    sg_rt_mean_repeat = df_source[f"snap_game_true_pos_rt_avrg.{repeat}"].mean() 
    sg_rt_std_repeat = df_source[f"snap_game_true_pos_rt_avrg.{repeat}"].std()
    assignment[f'z_snap_game_true_pos_rt_avrg.{repeat}'] = (df_source[f"snap_game_true_pos_rt_avrg.{repeat}"] - sg_rt_mean_0)/sg_rt_std_0 
    
     # generate Global_v00 = (zDSTf_v00 + zDSTb_v00 - zTMTa_v00 - zTMTb_v00) / 4
    assignment["global.0"] = (- assignment['z_snap_game_true_pos_rt_avrg.0'] + weight_mem * assignment[f'z_{mem_experiment_full}.0'])/2.0
    sum_global_0 = assignment["global.0"].sum()
    mean_global_0 = assignment["global.0"].mean()
    std_global_0 = assignment["global.0"].std()
    assignment['z_global.0'] = (assignment["global.0"] - mean_global_0)/std_global_0

    assignment[f"global.{repeat}"] = (- assignment[f'z_snap_game_true_pos_rt_avrg.{repeat}'] + weight_mem * assignment[f'z_{mem_experiment_full}.{repeat}'])/2.0 
    assignment[f"z_global.{repeat}"] = (assignment[f"global.{repeat}"] - mean_global_0)/std_global_0
    
   
    
    assignment[f"z_change_{mem_experiment_full}.0.{repeat}"] = assignment[f'z_{mem_experiment_full}.{repeat}'] - assignment[f'z_{mem_experiment_full}.0']
    assignment[f"z_change_snap_game_true_pos_rt_avrg.0.{repeat}"] = assignment[f'z_snap_game_true_pos_rt_avrg.{repeat}'] - assignment['z_snap_game_true_pos_rt_avrg.0']
    df_source = df_source.assign(**assignment) 
    return df_source    

In [ ]:
if memory_experiment == "pm":
    memory_experiment = f"log_{memory_experiment}"
    memory_experiment_full_name = f"log_{memory_experiment_full_name}"
    

In [ ]:
df = adding_rv_scores(df, memory_experiment_full_name, second_visit)
df_imputed = adding_rv_scores(df_imputed, memory_experiment_full_name, second_visit)


In [ ]:

# these files are used to test validator.R
df.to_csv(f"{output_dir}/test_non_na.csv", sep=',')
df_imputed.to_csv(f"{output_dir}/test_ukbb_imputed_{memory_experiment}_sg_0_{second_visit}_rf_instance_1.csv", sep=',')

In [ ]:
# describe population
def describe_population(df_, descr): 
    print(descr)
    comorbidity = ["Diabetes_2.0",  "Dyslipidemia.0", "Depression.0", "Hypertension.0"]
    n_comor = {}
    percentage_comor = {}
    n_comor_female = {}
    percentage_comor_female = {}
    for comor in comorbidity:
        if comor in df_.columns: 
            n_comor[comor] = df_[comor].sum()
            percentage_comor[comor] = (float(n_comor[comor])*100)/float(len(df_))
            print(f"The % of participants with {comor} is {percentage_comor[comor]}")
            if "gender.0" in df_.columns:
                n_comor_female[comor] = (df_[comor] + df_["gender.0"]).eq(2.0).sum()
                percentage_comor_female[comor] = (float(n_comor_female[comor])*100)/float(len(df_))
                print(f"The % of female participants with {comor} is {percentage_comor_female[comor]}")

In [ ]:
describe_population(df, "check up df with no NA")

In [ ]:
describe_population(df_imputed, "df with imputations")

In [ ]:
# function to add interactions
def add_interactions(df_, interaction_lists, interaction_pairs, descr):
    if len(interaction_lists) == 0 and len(interaction_pairs) == 0:
        return df_, []
    print(f"adding interaction  features to {descr}")
    assignment = {}
    interaction_cols = []

    for int_1 in interaction_lists[0]:
        print(int_1)
        for int_2 in interaction_lists[1]:
            print(int_2)
            new_fld = f"{int_1}_x_{int_2}"
            if new_fld not in df_.columns:
                assignment[new_fld] = df_[int_1] * df_[int_2]
                interaction_cols = [new_fld] + interaction_cols
    df_ = df_.assign(**assignment) 
     
    assignment = {}
    print(df_.columns)
    for interaction_pair in interaction_pairs:
        new_fld = f"{interaction_pair[0]}_x_{interaction_pair[1]}"
        if new_fld not in df_.columns:
            assignment[new_fld] = df_[interaction_pair[0]] * df_[interaction_pair[1]]
            print(new_fld)
            interaction_cols = [new_fld] + interaction_cols
    df_ = df_.assign(**assignment)   
    return df_, interaction_cols

In [ ]:
df, interaction_cols =  add_interactions(df, interaction_lists, interaction_pairs, "check up db with no NA")

In [ ]:
df_imputed, _  =  add_interactions(df_imputed, interaction_lists, interaction_pairs, "df with imputed")

In [ ]:
# add logarithms and interactions to the predictors list
predictors = predictors + log_cols + quad_cols + interaction_cols

for predictor in remove_original_features:
    if predictor in predictors:
        predictors.remove(predictor)
    
if (gender is not None) and ("gender.0") in predictors:
    predictors.remove("gender.0") 
    
    
print(predictors)

In [ ]:
# functions which runs lasso, build an ntermediate linear model by on features selected by lasso and 
# then build the final model on features significant by the inetrmediate model
def model_builder(df_source, predictors, outcome_var, result_dir, keep_cholesterols, hcd, run_lasso = True):
    
    if not os.path.exists(result_dir):
    # The directory does not exist, create it
        os.makedirs(result_dir)
        
    y = df_source[outcome_var]
    print(f"outcome: {y}")
    
    X = df_source[predictors]
   
    
    ############ LASSO ###### 
    if (run_lasso):
        
        
        scaler = StandardScaler()
        # Initialize LassoCV with 10-fold cross-validation
        lasso_cv = LassoCV(cv=100, random_state=42, max_iter=10000)
        pipeline = make_pipeline(scaler, lasso_cv)
        pipeline.fit(X, y)
    
    

        summary_text = f"Best alpha (regularization strength): {lasso_cv.alpha_}\n"
        summary_text += f"Intercept {lasso_cv.intercept_}\nCoefficients:\n"
        for idx, coef in enumerate(lasso_cv.coef_, start=1):
            summary_text += f"Feature {lasso_cv.coef_[idx]}: {coef}\n"

        
        # Saving to a file
        with open(f"{result_dir}/lasso_summary.txt", 'w') as lasso_file:
            lasso_file.write(summary_text)
        
    
    
    
        corrected_predictors = []
        for i in range(len(predictors)):
            if abs(lasso_cv.coef_[i]) >lasso_epsilon: # or (predictors[i] in simple_predictors):
                print(f"{predictors[i]}: {lasso_cv.coef_[i]}")
                corrected_predictors = corrected_predictors + [predictors[i]]
        
        if keep_cholesterols:
            if "ldl.0" in corrected_predictors and not ("hdl.0" in corrected_predictors):
                corrected_predictors = corrected_predictors  + ["hdl.0"]
            if "hdl.0" in corrected_predictors and not ("ldl.0" in corrected_predictors):
                corrected_predictors = corrected_predictors  + ["ldl.0"]  
            
   
        if len(corrected_predictors)== 0:
            print("All the coefficients are zeros")
            return None 
        
        significant_features = corrected_predictors
        ############ LINEAR MODEL AFTER LASSO ######
        
        '''
        X_corr = df_source[corrected_predictors]
        for clmn in  X_corr.columns:  
            X_corr[clmn] = (X_corr[clmn] - X_corr[clmn].mean())/X_corr[clmn].std()
    
        X_corr = sm.add_constant(X_corr)
        if hcd is not None:
            model = sm.OLS(y, X_corr).fit(cov_type=hcd)
        else:
            model = sm.OLS(y, X_corr).fit()
        
        
        with open(f"{result_dir}/after_lasso_model_summary.txt", 'w') as model_file:
        # Convert the summary to a string and write it to the file
            model_file.write(model.summary().as_text())
        
        ############ FINAL LINEAR MODEL  ###### 
    
   
        significant_features = model.pvalues[model.pvalues <= p_val].index.tolist()
        
        if 'const' in significant_features:
            significant_features.remove('const') '''
    else:
       significant_features = predictors
    
    
    
    
    X_corr_1 = df_source[significant_features]
    df_mean_std = pd.DataFrame(columns = ["predictor", "mean", "standard_deviation", "min", "max"] )
    i = 0
    for clmn in  X_corr_1.columns:  
        mean_ = X_corr_1[clmn].mean()
        std_ = X_corr_1[clmn].std()
        df_mean_std.loc[i] = [clmn, mean_, std_, X_corr_1[clmn].min(), X_corr_1[clmn].max()]
        i += 1
        X_corr_1[clmn] = (X_corr_1[clmn] - mean_)/std_
        
    
    df_mean_std.to_csv(f"{result_dir}/final_predictors_stat.csv", sep=',')
    
        
    X_corr_1 = sm.add_constant(X_corr_1) 
    if hcd is not None:
        model_1= sm.OLS(y, X_corr_1).fit(cov_type=hcd) 
    else:
        model_1= sm.OLS(y, X_corr_1).fit() 
    
    
    with open(f"{result_dir}/final_model_summary.txt", 'w') as model_1_file:
    # Convert the summary to a string and write it to the file
        model_1_file.write(model_1.summary().as_text())
    
    coefficients = model_1.params
    p_values = model_1.pvalues
    
    df_results = pd.DataFrame({'Coefficient': coefficients, 'P_value': p_values})
    
    df_results = df_results.reset_index().rename(columns={df_results.reset_index().columns[0]: 'predictor'})
    df_results = pd.merge(df_results, df_mean_std, on = "predictor", how = 'outer')
    
    
    df_results.to_csv(f"{result_dir}/final_model.csv", sep=',')
    return df_results

sum TMTa_v00 if TMTa_v00!=. & TMTa_v02!=.
scalar TMTamean_v00 = r(mean)
sum TMTa_v00 if TMTa_v00!=. & TMTa_v02!=.
scalar TMTaSD_v00 = r(sd)
generate zTMTa_v00 = (TMTa_v00-TMTamean_v00)/TMTaSD_v00 if TMTa_v00!=. & TMTa_v02!=.

sum TMTa_v02 if TMTa_v00!=. & TMTa_v02!=.
scalar TMTamean_v02 = r(mean)
sum TMTa_v02 if TMTa_v00!=. & TMTa_v02!=.
scalar TMTaSD_v02 = r(sd)
generate zTMTa_v02 = (TMTa_v02-TMTamean_v00)/TMTaSD_v00 if TMTa_v00!=. & TMTa_v02!=.
*--------

In [ ]:
# filetr predictors so that all the predictors with test names mentioned are removed
# came from the earlier experiemnts where the amount of tests performed prior visit 2 were taken into account
# possibly an obsolete step by no but let us keep it
predictors_global =  copy.deepcopy(predictors)
for predictor in predictors:
    if "snap_game" in predictor:
        predictors_global.remove(predictor)
    if "pairs_matching" in predictor:
        predictors_global.remove(predictor)

In [ ]:
def show_dependencies(df_, model_df, predictors_to_show, dir):
    
    if model_df is None:
        return
    
    if os.path.exists(dir):
        shutil.rmtree(dir)
    os.makedirs(dir)
    
    print("we are building graphs")
    
    for predictor in predictors_to_show:
        
        
        print(predictor)
       
        y = np.zeros(df_.shape[0])
        # 
        # aggregate all the terms containing pedictor 
        for feat in model_df['predictor']:
            if not predictor in feat:
                continue
            if not feat in df_.columns:
                continue
            print(feat)
            
            # prepare the feature
            match feat:
                case _ if feat == predictor:
                    fun_pred = df_[predictor]
                case _ if feat == f"log_{predictor}":
                    fun_pred = np.log1p(df_[predictor])
                case _ if feat == f"quadr_{predictor}":
                    fun_pred = np.square(df_[predictor])
                case _ if feat == f"quadr_log_{predictor}":
                    fun_pred = np.square(np.log1p(df_[predictor]))
                case _:
                    fun_pred = None
                    print(f"error, non existing predictor {feat}")
            # standartise the feature
            fun_pred = (fun_pred - fun_pred.mean())/fun_pred.std()
            
            
            
            # find the coefficient
            row_feat = model_df[model_df['predictor'] == feat].iloc[0]
            coeff =  row_feat["Coefficient"]
            
            y = y + coeff *fun_pred
        
        # x_pred = (df_[predictor]    - df_[predictor].mean()) /  df_[predictor].std()
        # x = np.linspace(min(x_pred), max(x_pred), 100)  
          
        are_all_zeros = np.all(y == 0)        
        if not are_all_zeros:
            x = df_[predictor]
        
            plt.scatter(x, y)
            plt.xlabel(predictor)
            plt.ylabel('outcome')
            plt.legend()
            plt.grid(True)

            # Save the plot as a PNG file
            plt.savefig(f"{dir}/{predictor}.png", format='png', dpi=300)
            plt.show()

In [ ]:

outcomes = [f"z_global.{second_visit}", f"z_change_{memory_experiment_full_name}.0.{second_visit}",  f"z_change_snap_game_true_pos_rt_avrg.0.{second_visit}"]
model_table_no_na = {}
model_table_imputed = {}
model_table_no_na_simple = {}
model_table_imputed_simple = {}
for outcome in outcomes:
    print(outcome)
    
    # main
    
    print('main no na')
    model_table_no_na[outcome] = model_builder(df, predictors_global, outcome, f"{output_dir}/no_na_{outcome}", keep_ldl_hdl, HSCD)
    show_dependencies(df, model_table_no_na[outcome], predictors_to_plot, f"{output_dir}/no_na_{outcome}/figures")
    print('main imputed')
    model_table_imputed[outcome] = model_builder(df_imputed, predictors_global, outcome, f"{output_dir}/imputed_{outcome}", keep_ldl_hdl, HSCD)
    show_dependencies(df_imputed, model_table_imputed[outcome], predictors_to_plot, f"{output_dir}/imputed_{outcome}/figures")
    
    if compute_simple_models:
        # simple
        
        print('simple no na')
        model_table_no_na_simple[outcome] = model_builder(df, simple_predictors, outcome, f"{output_dir}/simple_no_na_{outcome}", keep_ldl_hdl, HSCD, False)
        
        print('simple imputed')
        model_table_imputed_simple[outcome] = model_builder(df_imputed, simple_predictors, outcome, f"{output_dir}/simple_imputed_{outcome}", keep_ldl_hdl, HSCD, False)
        